# Orbit Determination using Gauss Method (WIP)

This tutorial demonstrates how to implement Gauss' method of orbit determination from three position vectors. 

Dedicated optimization solvers will be implemented directly in OSTk soon to be used for convex optimization problems.

## Setup

In [ ]:
import datetime

import numpy as np
import pandas as pd
import scipy.optimize

import plotly.graph_objs as go
from plotly.subplots import make_subplots

from ostk.mathematics.objects import RealInterval

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.units import Time
from ostk.physics.units import Derived
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Velocity
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate.spherical import AER
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Kepler
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE
from ostk.astrodynamics import Access
from ostk.astrodynamics.access import Generator as AccessGenerator

In [ ]:
def convert_state (instant, state):
    
    lla = LLA.cartesian(state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(), Earth.equatorial_radius, Earth.flattening)
    
    return [
        repr(instant),
        float(instant.get_modified_julian_date(Scale.UTC)),
        *state.get_position().get_coordinates().transpose().tolist(),
        *state.get_velocity().get_coordinates().transpose().tolist(),
        float(lla.get_latitude().in_degrees()),
        float(lla.get_longitude().in_degrees()),
        float(lla.get_altitude().in_meters())
    ]

def generate_orbit_df (orbit, interval, step):
    
    if isinstance(interval, tuple):
        interval = Interval.closed(interval[0], interval[1]).generate_grid(step)
    
    orbit_data = [
        convert_state(instant, orbit.get_state_at(instant))
        for instant in interval
    ]

    return pd.DataFrame(
        orbit_data,
        columns = [
            '$Time^{UTC}$',
            '$MJD^{UTC}$',
            '$x_{x}^{ECI}$',
            '$x_{y}^{ECI}$',
            '$x_{z}^{ECI}$',
            '$v_{x}^{ECI}$',
            '$v_{y}^{ECI}$',
            '$v_{z}^{ECI}$',
            '$Latitude$',
            '$Longitude$',
            '$Altitude$'
        ]
    )

def x_ECI_from_ECEF (x_ECEF, instant):
    return Position.meters(x_ECEF, Frame.ITRF()).in_frame(Frame.GCRF(), instant).get_coordinates()

def generate_orbit (epoch, x, v):
    
    coe = COE.cartesian(
        [
            Position.meters(x, Frame.GCRF()),
            Velocity.meters_per_second(v, Frame.GCRF())
        ],
        Earth.gravitational_parameter
    )
    
    return Orbit(Kepler(coe, epoch, earth, Kepler.PerturbationType.J2), earth)
    # J4 might be broken: result being very different from J2.

---

In [ ]:
analysis_duration = Duration.days(1.0)
step = Duration.minutes(1.0)

---

Bootstrap environment:

In [ ]:
earth = Environment.default().access_celestial_object_with_name('Earth')

## Reference Trajectory

In [ ]:
tle = TLE(
    '1 99994U          21181.85503887  .00000000  00000-0  00000-0 0 00000',
    '2 99994 097.5068 310.6402 0011081 218.2042 145.7731 15.12723100000014'
)

ref_orbit = Orbit(SGP4(tle), earth)

In [ ]:
tle.get_epoch()

In [ ]:
n = 3.0 # ~ 120 deg spaced-out sample points

In [ ]:
period = 1 / tle.get_mean_motion().in_unit(Derived.Unit.angular_velocity(Angle.Unit.Revolution, Time.Unit.Minute))

In [ ]:
print(f'1/{n} Period: {period / n} (min)')

---

## Configuration

Define three instants:

In [ ]:
start_instant = Instant.date_time(DateTime.parse('2021-07-02T18:00:00.000', DateTime.Format.ISO8601), Scale.UTC)

# t_1 = start_instant
# t_2 = start_instant + Duration.minutes(period / n)
# t_3 = start_instant + Duration.minutes(period / n) * 2

t_1 = Instant.date_time(datetime.datetime(2021, 7, 2, 9, 28, 46, 712, tzinfo = datetime.timezone.utc), Scale.UTC)
t_2 = Instant.date_time(datetime.datetime(2021, 7, 2, 11, 4, 52, 712, tzinfo = datetime.timezone.utc), Scale.UTC)
t_3 = Instant.date_time(datetime.datetime(2021, 7, 2, 13, 21, 16, 712, tzinfo = datetime.timezone.utc), Scale.UTC)

In [ ]:
print(f't_1: {t_1.get_date_time(Scale.UTC)}')
print(f't_2: {t_2.get_date_time(Scale.UTC)}')
print(f't_3: {t_3.get_date_time(Scale.UTC)}')

Define three position vectors (ECEF):

In [ ]:
# x_ECEF_1 = ref_orbit.get_state_at(t_1).get_position().in_frame(Frame.ITRF(), t_1).get_coordinates()
# x_ECEF_2 = ref_orbit.get_state_at(t_2).get_position().in_frame(Frame.ITRF(), t_2).get_coordinates()
# x_ECEF_3 = ref_orbit.get_state_at(t_3).get_position().in_frame(Frame.ITRF(), t_3).get_coordinates()

# x_ECEF_1 = np.array([-1815703.172893414,704970.0470229279,6621051.463822694])*1e3 # [m]
# x_ECEF_2 = np.array([-2054436.5277931439,985835.8528682495,6514351.320023936])*1e3 # [m]
# x_ECEF_1 = np.array([137602.2258204725,1182569.6340145601,-6805298.931909148])*1e3 # [m]

Convert to ECI:

In [ ]:
# x_1 = x_ECI_from_ECEF(x_ECEF_1, t_1)
# x_2 = x_ECI_from_ECEF(x_ECEF_2, t_2)
# x_3 = x_ECI_from_ECEF(x_ECEF_3, t_3)

x_1 = np.array([-1815703.172893414,704970.0470229279,6621051.463822694]) # [m]
x_2 = np.array([-2054436.5277931439,985835.8528682495,6514351.320023936]) # [m]
x_3 = np.array([137602.2258204725,1182569.6340145601,-6805298.931909148]) # [m]

In [ ]:
print(f'x_ECI_1: {x_1 / 1e3} (km)')
print(f'x_ECI_2: {x_2 / 1e3} (km)')
print(f'x_ECI_3: {x_3 / 1e3} (km)')

Generate a time grid:

In [ ]:
instants = Interval.closed(t_1, t_3).generate_grid(step)

if t_2 not in instants:
    instants.append(t_2)
    
if t_3 not in instants:
    instants.append(t_3)

instants.sort()

assert instants[0] == t_1
assert t_2 in instants
assert instants[-1] == t_3

---

## Derivation of Orbital Elements

Derive classical orbital elements from three position vectors using Gibbs' method. Ref: [H. Curtis, Orbital Mechanics for Engineering Students](http://www.nssc.ac.cn/wxzygx/weixin/201607/P020160718380095698873.pdf) (algorithm and 5.1).

Compute cross products and verify that vectors are coplanar:

In [ ]:
c_12 = np.cross(x_1, x_2)
c_23 = np.cross(x_2, x_3)
c_31 = np.cross(x_3, x_1)

x_1_hat = x_1 / np.linalg.norm(x_1)
c_23_hat = c_23 / np.linalg.norm(c_23)

dot_product = abs(np.dot(x_1_hat, c_23_hat))

print(f'Coplanarity condition: dot product = {dot_product}.')
assert dot_product < 1e-2, 'The three position vectors are not coplanar.'

Compute estimate $\bf{v_2}^{ECI}_{rough}$:

In [ ]:
n = np.linalg.norm(x_1) * c_23 + np.linalg.norm(x_2) * c_31 + np.linalg.norm(x_3) * c_12
d = c_12 + c_23 + c_31
s = x_1 * (np.linalg.norm(x_2) - np.linalg.norm(x_3)) + x_2 * (np.linalg.norm(x_3) - np.linalg.norm(x_1)) + x_3 * (np.linalg.norm(x_1) - np.linalg.norm(x_2))

mu = float(Earth.gravitational_parameter.in_unit(Derived.Unit.gravitational_parameter(Length.Unit.Meter, Time.Unit.Second)))

v_2_rough = np.sqrt(mu / (np.linalg.norm(n) * np.linalg.norm(d))) * (np.cross(d, x_2, -1) / np.linalg.norm(x_2) + s)

In [ ]:
print(f'x_2 = {x_2 / 1e3} (km)')
print(f'v_2_rough = {v_2_rough / 1e3} (km/s)')

---

## Optimization

In [ ]:
optimization_interval = (t_1, t_1 + analysis_duration)

In [ ]:
ref_orbit_df = generate_orbit_df(ref_orbit, optimization_interval, step)

In [ ]:
def d_orbit (v):
    
    orbit = generate_orbit(
        epoch = t_2,
        x = x_2,
        v = v
    )
    
    orbit_df = generate_orbit_df(orbit, optimization_interval, step)
    
    dx = ref_orbit_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy() \
    - orbit_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy()
    
    dx_ECI_norm = np.linalg.norm(
        dx,
        axis = 1,
    )

    return dx_ECI_norm

This notebook uses scipy and existing solvers. Soon OSTk will feature those directly !

In [ ]:
res = scipy.optimize.least_squares(
    d_orbit,
    v_2_rough
)

In [ ]:
v_2 = res.x

In [ ]:
orbit = generate_orbit(
    epoch = t_2,
    x = x_2,
    v = v_2
)

In [ ]:
print(f'x_2 = {x_2 / 1e3} (km)')
print(f'v_2 = {v_2 / 1e3} (km/s)')

In [ ]:
validation_interval = (t_1 + analysis_duration * 1, t_1 + analysis_duration * 2)

In [ ]:
ref_orbit_df = generate_orbit_df(ref_orbit, validation_interval, step)
orbit_df = generate_orbit_df(orbit, validation_interval, step)

dx_ECI_norm = np.linalg.norm(
    ref_orbit_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy()
    - orbit_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy(),
    axis = 1
) / 1e3

dv_ECI_norm = np.linalg.norm(
    ref_orbit_df[['$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$']].to_numpy()
    - orbit_df[['$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$']].to_numpy(),
    axis = 1
) / 1e3

print(f'Min. |Δx_ECI|: {min(dx_ECI_norm):.3f} [km]')
print(f'Avg. |Δx_ECI|: {np.mean(dx_ECI_norm):.3f} [km]')
print(f'Max. |Δx_ECI|: {max(dx_ECI_norm):.3f} [km]')
print('\n')
print(f'Min. |Δv_ECI|: {min(dv_ECI_norm):.3f} [km/s]')
print(f'Avg. |Δv_ECI|: {np.mean(dv_ECI_norm):.3f} [km/s]')
print(f'Max. |Δv_ECI|: {max(dv_ECI_norm):.3f} [km/s]')

def mean(x):
    return np.mean(x) * np.ones(np.shape(x))

data = [
    dx_ECI_norm,
    dv_ECI_norm
]

figure = go.FigureWidget(make_subplots(rows = len(data), cols = 1))

for (i, d) in enumerate(data):

    figure.append_trace(
        go.Scatter(
            y = d,
            mode = 'lines+markers',
            marker_size = 3,
            marker_symbol = 'cross',
            line_width = 1,
        ),
        row = i + 1,
        col = 1
    )

    figure.append_trace(
        go.Scatter(
            y = mean(d),
            mode = 'lines',
            line = dict(
                color = 'rgb(0, 0, 0)',
                width = 1
            ),
        ),
        row = i + 1,
        col = 1
    )

figure.update_xaxes(title_text = 'Time (min)', row = len(data), col = 1)
figure.update_yaxes(title_text = '|Δx_ECI| (km)', row = 1, col = 1)
figure.update_yaxes(title_text = '|Δv_ECI| (km/s)', row = 2, col = 1)

figure.update_layout(showlegend = False)

figure.show()

---

## Visualization

Orbit:

In [ ]:
orbit_df.head()

2D plot, over **World Map**:

In [ ]:
figure = go.Figure(
    data = [
        go.Scattergeo(
            lon = orbit_df['$Longitude$'],
            lat = orbit_df['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 1,
                color = 'gray'
            )
        )
    ],
    layout = go.Layout(
        title = None,
        showlegend = False,
        height=1000,
        geo = go.layout.Geo(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)'
        )
    )
)

figure.show()

3D plot, in **Earth Fixed** frame:

In [ ]:
figure = go.Figure(
    data = [
        go.Scattergeo(
            lon = orbit_df['$Longitude$'],
            lat = orbit_df['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 1,
                color = 'rgba(255, 0, 0, 0.5)'
            )
        )
    ],
    layout = go.Layout(
        title = None,
        showlegend = False,
        width = 800,
        height = 800,
        geo = go.layout.Geo(
            showland = True,
            showlakes = True,
            showcountries = False,
            showocean = True,
            countrywidth = 0.0,
            landcolor = 'rgb(100, 100, 100)',
            lakecolor = 'rgb(240, 240, 240)',
            oceancolor = 'rgb(240, 240, 240)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            lonaxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            ),
            lataxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            )
        )
    )
)

figure.show()

---

Generate a new TLE using STK.

In [ ]:
...

Define obtained TLE:

In [ ]:
new_tle = TLE(
    '1 99993U          21181.86036100  .00000000  00000-0  00000-0 0 00009',
    '2 99993 097.5023 310.6314 0013147 265.4618 127.4558 15.12561269000014'
)

or patch a TLE from estimated state vector, in this notebook:

In [ ]:
def epoch_from_instant (instant):
    
    date_time = instant.get_date_time(Scale.UTC).replace(tzinfo = datetime.timezone.utc)
    
    year = int(f'{date_time.year}'[2:])
    year_day = date_time.timetuple().tm_yday
    day_fraction = (date_time.hour * 3600e6 + date_time.minute * 60e6 + date_time.second * 1e6 + date_time.microsecond) / 86400e6
    
    return year * 1000 + year_day + day_fraction

def update_checksum (tle_line: str) -> str:

    assert len(tle_line) == 69

    tle_line_digits: str = ''.join(filter(str.isdigit, tle_line[0:68]))

    checksum: int = sum(int(a) for a in tle_line_digits) + tle_line[0:68].count('-')

    return tle_line[0:68] + str(checksum % 10)

def patch_tle_from_coe (coe):
    
    line1 = '1 99994U          21176.78890016  .00000000  00000-0  00000-0 0 00009'
    line2 = '2 99994 097.4948 296.8998 0009655 246.0231 250.6116 15.12610161000013'
    
    line1 = list(line1)
    line2 = list(line2)
    
    line1[18:32] = f'{epoch_from_instant(t_2):.8f}'.zfill(14)
    
    line2[8:16] = f'{float(coe.get_inclination().in_degrees()):.4f}'.zfill(8)
    line2[17:25] = f'{float(coe.get_raan().in_degrees()):.4f}'.zfill(8)
    line2[26:33] = f'{float(coe.get_eccentricity()):.7f}'[2:].zfill(7)
    line2[34:42] = f'{float(coe.get_aop().in_degrees()):.4f}'.zfill(8) 
    line2[43:51] = f'{float(coe.get_mean_anomaly().in_degrees()):.4f}'.zfill(8) 
    
    # coe.get_mean_motion might be broken
    sma = float(coe.get_semi_major_axis().in_meters())
    mean_motion = np.sqrt(mu / sma**3) * 86400 / (2 * np.pi)
    line2[52:63] = f'{mean_motion:.8f}'.zfill(11)
    
    line1 = ''.join(line1)
    line2 = ''.join(line2)

    line1 = update_checksum(line1)
    line2 = update_checksum(line2)
    
    return (line1, line2)

In [ ]:
def d_tle (v):
    
    coe = COE.cartesian(
        [
            Position.meters(x_2, Frame.GCRF()),
            Velocity.meters_per_second(v, Frame.GCRF())
        ],
        Earth.gravitational_parameter
    )
    
    patched_tle = patch_tle_from_coe(coe)
    
    tle = TLE(
        patched_tle[0],
        patched_tle[1]
    )
    
    orbit = Orbit(SGP4(tle), earth)

    try:
        orbit_df = generate_orbit_df(orbit, optimization_interval, step)
    except RuntimeError:
        return 1e15
    
    dx = ref_orbit_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy() \
    - orbit_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy()
    
    dx_ECI_norm = np.linalg.norm(
        dx,
        axis = 1,
    )

    return dx_ECI_norm

In [ ]:
# res = scipy.optimize.least_squares(
#     d_tle,
#     v_2
# )

In [ ]:
# coe = COE.cartesian(
#     [
#         Position.meters(x_2, Frame.GCRF()),
#         Velocity.meters_per_second(res.x, Frame.GCRF())
#     ],
#     Earth.gravitational_parameter
# )

# patched_tle = patch_tle_from_coe(coe)

# new_tle = TLE(
#     patched_tle[0],
#     patched_tle[1]
# )

# print(new_tle)

Compare trajectories using initial and new TLE.

In [ ]:
ref_orbit_df = generate_orbit_df(ref_orbit, validation_interval, step)

new_tle_orbit = Orbit(SGP4(new_tle), earth)

new_tle_orbit_df = generate_orbit_df(new_tle_orbit, validation_interval, step)

dx_ECI_norm = np.linalg.norm(
    ref_orbit_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy()
    - new_tle_orbit_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy(),
    axis = 1
) / 1e3

dv_ECI_norm = np.linalg.norm(
    ref_orbit_df[['$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$']].to_numpy()
    - new_tle_orbit_df[['$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$']].to_numpy(),
    axis = 1
) / 1e3

print(f'Min. |Δx_ECI|: {min(dx_ECI_norm):.3f} [km]')
print(f'Avg. |Δx_ECI|: {np.mean(dx_ECI_norm):.3f} [km]')
print(f'Max. |Δx_ECI|: {max(dx_ECI_norm):.3f} [km]')
print('\n')
print(f'Min. |Δv_ECI|: {min(dv_ECI_norm):.3f} [km/s]')
print(f'Avg. |Δv_ECI|: {np.mean(dv_ECI_norm):.3f} [km/s]')
print(f'Max. |Δv_ECI|: {max(dv_ECI_norm):.3f} [km/s]')

def mean(x):
    return np.mean(x) * np.ones(np.shape(x))

data = [
    dx_ECI_norm,
    dv_ECI_norm
]

figure = go.FigureWidget(make_subplots(rows = len(data), cols = 1))

for (i, d) in enumerate(data):

    figure.append_trace(
        go.Scatter(
            y = d,
            mode = 'lines+markers',
            marker_size = 3,
            marker_symbol = 'cross',
            line_width = 1,
        ),
        row = i + 1,
        col = 1
    )

    figure.append_trace(
        go.Scatter(
            y = mean(d),
            mode = 'lines',
            line = dict(
                color = 'rgb(0, 0, 0)',
                width = 1
            ),
        ),
        row = i + 1,
        col = 1
    )

figure.update_xaxes(title_text = 'Time (min)', row = len(data), col = 1)
figure.update_yaxes(title_text = '|Δx_ECI| (km)', row = 1, col = 1)
figure.update_yaxes(title_text = '|Δv_ECI| (km/s)', row = 2, col = 1)

figure.update_layout(showlegend = False)

figure.show()

---